<a href="https://colab.research.google.com/github/Tsarh/python/blob/main/traitement_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install firebase-admin pandas
!pip install firebase-admin
!pip install firebase-admin pandas matplotlib nltk
!pip install pandas numpy firebase-admin
!pip install spacy
!pip install --upgrade sympy
!pip install --upgrade transformers
!pip install --upgrade torch torchvision torchaudio
!pip install spacy-transformers transformers
!pip install spacy-transformers --upgrade

In [ ]:
# !python -m spacy download fr_core_news_sm
!python -m spacy download en_core_web_sm
!pip install ipdb

In [ ]:
# @title Méthode n°2
from collections import defaultdict
import firebase_admin
from firebase_admin import credentials, db
import torch
from transformers import BertTokenizer, BertForTokenClassification
import spacy
import numpy as np
from sklearn.cluster import KMeans
import datetime
from google.colab import userdata
import re
import ipdb

In [4]:
from google.colab import files
files.upload()

{}

define product idea

In [5]:
import random

# Fonction pour associer des définitions aux idées de produits
def define_product_idea(idea):
    facile_a_utiliser = [
        "Un produit offrant une interface claire et des fonctionnalités accessibles sans nécessiter une formation préalable.",
        "Un outil dont les utilisateurs peuvent comprendre et utiliser les fonctions principales dès le premier contact.",
        "Un produit qui réduit la complexité au minimum pour garantir une expérience utilisateur fluide.",
        "Un dispositif ou un logiciel optimisé pour permettre une navigation naturelle et sans confusion.",
        "Un produit qui guide l’utilisateur à travers ses fonctionnalités sans qu’un manuel soit nécessaire.",
        "Un produit conçu pour être intuitif et simple d'utilisation, même pour les débutants.",
        "Un système pensé pour éliminer les barrières techniques et favoriser l’efficacité immédiate.",
        "Un produit où les actions essentielles sont évidentes et nécessitent peu de réflexion.",
        "Une solution où les utilisateurs atteignent leurs objectifs rapidement grâce à une conception ergonomique.",
        "Un produit accessible à tout public, indépendamment de son niveau de compétence technique.",
        "Un outil offrant une expérience utilisateur cohérente et sans ambiguïtés à chaque étape.",
        "Un produit où chaque fonctionnalité est présentée de manière logique et compréhensible.",
        "Une solution développée avec des éléments familiers pour garantir un apprentissage intuitif.",
        "Un produit conçu pour minimiser les erreurs grâce à une navigation et des instructions claires.",
        "Un outil dont l’utilisation repose sur des actions instinctives et prévisibles.",
        "Une interface qui met l’accent sur la clarté visuelle et les interactions simplifiées.",
        "Un produit offrant un processus d’apprentissage rapide grâce à des interfaces conviviales.",
        "Une solution pensée pour que les utilisateurs accomplissent leurs tâches avec un minimum d’efforts.",
        "Un produit qui anticipe les besoins des utilisateurs en rendant les fonctions essentielles facilement accessibles.",
        "Un outil qui élimine la frustration en réduisant le temps nécessaire pour comprendre son fonctionnement.",
        "Un produit qui rend les tâches complexes simples grâce à un design centré sur l’utilisateur."
    ]

    produit_economique = [
        "Un produit abordable qui offre un bon rapport qualité-prix.",
        "Un bien ou service qui permet de réaliser des économies grâce à un coût d'achat ou d'utilisation réduit.",
        "Un produit conçu pour répondre aux besoins essentiels à un prix accessible.",
        "Un bien dont le prix est inférieur à la moyenne du marché tout en maintenant une qualité acceptable.",
        "Un produit pensé pour offrir une fonctionnalité maximale avec un minimum de dépenses.",
        "Un bien fabriqué en optimisant les coûts de production afin d’être vendu à un tarif compétitif.",
        "Un article destiné aux consommateurs cherchant à minimiser leurs dépenses sans sacrifier la praticité.",
        "Un produit qui allie simplicité, efficacité et un faible coût global d'acquisition et d'entretien.",
        "Un bien consommable qui privilégie l'essentiel au luxe, en ciblant une clientèle soucieuse de son budget.",
        "Un produit qui offre une solution accessible aux défis financiers des consommateurs.",
        "Un bien ou service standardisé, conçu pour être fonctionnel tout en restant financièrement accessible au plus grand nombre."
    ]

    qualite = [
        "Les caractéristiques essentielles qui définissent la nature d'une chose ou d'un être.",
        "Le niveau d'excellence ou de performance d'un produit ou service.",
        "L'appréciation subjective d'un produit, service ou individu basée sur ses avantages ou ses mérites.",
        "L'adéquation d'un produit ou service aux normes, exigences ou attentes spécifiées.",
        "Un trait ou une caractéristique qui distingue un objet, une personne ou une action des autres.",
        "Le niveau auquel un produit ou service répond aux attentes des utilisateurs ou clients.",
        "Une vertu ou une disposition positive dans le comportement ou le caractère d'une personne.",
        "Une mesure ou un standard utilisé pour juger ou comparer la valeur ou l'efficacité de quelque chose.",
        "La capacité d'un produit ou service à rester performant et utile sur une longue période.",
        "La manière dont les différentes composantes d'un objet ou système fonctionnent ensemble pour atteindre un objectif.",
        "Un avantage ou une distinction qui rend quelque chose meilleur par rapport à des alternatives."
    ]


    outil = [
        "Un dispositif utilisé pour accomplir une tâche spécifique.",
        "Un instrument conçu pour faciliter la réalisation d'un travail précis.",
        "Un objet ou un dispositif permettant d'exécuter une action ou une fonction particulière.",
        "Un équipement physique ou numérique utilisé pour résoudre un problème ou produire un résultat.",
        "Un moyen ou une ressource servant à améliorer l'efficacité d'une tâche.",
        "Un dispositif pratique utilisé pour manipuler, transformer ou créer quelque chose.",
        "Une ressource matérielle ou immatérielle mise en œuvre pour atteindre un objectif.",
        "Un appareil ou un mécanisme simplifiant ou automatisant un processus.",
        "Un élément conçu pour servir dans un contexte technique ou professionnel.",
        "Un objet, un logiciel ou une technique permettant de réaliser un travail particulier avec précision.",
        "Un support ou une aide permettant d'accomplir une tâche plus efficacement."
    ]

    appareil = [
        "Un équipement électronique ou mécanique pour une utilisation quotidienne.",
        "Ensemble de pièces ou de dispositifs conçus pour accomplir une fonction spécifique.",
        "Instrument ou machine utilisée pour effectuer une tâche technique ou scientifique.",
        "Dispositif conçu pour répondre à un besoin pratique ou utilitaire dans divers domaines, comme la maison, l’industrie ou la médecine.",
        "Structure organisée composée d'éléments interdépendants pour remplir une fonction donnée, par exemple dans le corps humain ou une organisation.",
        "Objet mécanique, électrique ou numérique destiné à faciliter ou automatiser une action."
    ]

    technologie = [
        "Un produit intégrant des avancées technologiques pour améliorer l'efficacité ou le confort.",
        "L'ensemble des connaissances, outils et méthodes utilisés pour concevoir, produire et améliorer des biens ou des services.",
        "L'application pratique des sciences et des découvertes pour répondre à des besoins humains ou résoudre des problèmes.",
        "Un domaine en constante évolution visant à transformer les idées et les innovations en solutions tangibles pour améliorer la vie quotidienne.",
        "L'étude et l'utilisation des procédés techniques pour créer ou améliorer des systèmes, machines ou logiciels.",
        "Un moyen d'adapter les ressources disponibles pour développer des solutions innovantes et accessibles, adaptées à un environnement donné."
    ]

    accessoire = [
        "Un objet complémentaire utilisé pour améliorer ou personnaliser une expérience.",
        "Un élément secondaire ou décoratif qui accompagne un objet principal pour en augmenter l’utilité ou l’esthétique.",
        "Un objet ou une pièce ajoutée à un équipement principal pour élargir ses fonctionnalités ou répondre à des besoins spécifiques.",
        "Tout élément qui, bien que non essentiel, apporte une valeur ajoutée ou complète un ensemble ou une tenue."
    ]

    usure = [
        "L'affaiblissement progressif d'un matériau dû à des contraintes mécaniques répétées.",
        "La dégradation d'un produit avec le temps ou son utilisation fréquente.",
        "La perte de qualité ou de performance d'un objet après une utilisation prolongée.",
        "L'érosion naturelle causée par des facteurs externes comme le vent, l'eau ou le frottement.",
        "La détérioration morale ou psychologique d'une personne sous l'effet du stress ou de la fatigue.",
        "La diminution de la valeur ou de l'efficacité d'un système ou d'un processus au fil du temps.",
        "L'altération des propriétés physiques d'une surface due à des frottements répétés.",
        "La fatigue des matériaux sous l'effet de charges cycliques ou continues.",
        "La dégradation des relations humaines causée par des conflits ou des incompréhensions répétés.",
        "La perte d'énergie ou de motivation après un effort prolongé ou intense.",
        "L'effritement ou la désagrégation d'une structure due à des conditions environnementales défavorables."
    ]

    prix = [
        "Le coût ou la valeur monétaire d'un produit ou service.",
        "Le montant d'argent demandé ou payé pour acquérir un bien ou un service.",
        "La somme fixée pour l'échange d'une marchandise ou d'une prestation.",
        "La valeur marchande attribuée à un objet ou à une offre commerciale.",
        "Le tarif appliqué à un produit en fonction de l'offre et de la demande.",
        "L'expression monétaire de la valeur perçue d'un article ou d'un service.",
        "Le coût financier nécessaire pour obtenir quelque chose en échange.",
        "La quantification en argent d'un bien ou d'une utilité proposée.",
        "Le montant qu'un acheteur doit débourser pour obtenir un produit.",
        "La représentation économique de la valeur d'un objet ou d'une action.",
        "Le chiffre qui reflète le coût ou la valeur d'un élément sur un marché."
    ]

    avis = [
        "Un retour ou une opinion laissée par un utilisateur à propos d'un produit ou service.",
        "Une évaluation ou un jugement porté sur quelque chose, basé sur une expérience ou une observation.",
        "Une déclaration écrite ou orale exprimant un point de vue personnel sur un sujet, un objet ou une situation.",
        "Un commentaire formulé par une personne pour partager son sentiment ou son opinion sur un service, un produit ou une idée.",
        "Une recommandation ou une critique émise par un individu après avoir testé ou utilisé quelque chose.",
        "Un témoignage subjectif qui reflète l'impression ou la satisfaction d'une personne vis-à-vis d'une expérience vécue.",
        "Une réaction personnelle exprimée publiquement ou en privé pour donner son point de vue sur un sujet donné.",
        "Un feedback constructif ou informatif fourni par un utilisateur pour aider à améliorer ou à juger une offre.",
        "Une appréciation ou une notation attribuée à quelque chose, souvent basée sur des critères personnels ou objectifs.",
        "Un partage d'expérience sous forme de commentaire, visant à informer ou à influencer les décisions des autres.",
        "Une expression de satisfaction, d'insatisfaction ou de neutralité concernant un produit, un service ou une situation.",
    ]
    definitions = {
          # Produits et caractéristiques générales
          "produit facile à utiliser": random.choice(facile_a_utiliser),
          "produit économique": random.choice(produit_economique),
          "qualité": random.choice(qualite),
          "outil": random.choice(outil),
          "appareil": random.choice(appareil),
          "technologie": random.choice(technologie),
          "accessoire": random.choice(accessoire),
          "usure": random.choice(usure),
          "prix": random.choice(prix),
          "innovation": "L'introduction de nouvelles idées, fonctionnalités ou technologies dans un produit.",
          "marque": "Une entreprise ou un label spécifique connu pour ses produits de qualité.",
          "polyvalence": "La capacité d'un produit à s'adapter à différents usages ou contextes.",
          "performance": "La capacité d'un produit ou d'un outil à accomplir efficacement une tâche.",
          "fiabilité": "La capacité d'un produit à fonctionner sans problème sur une période prolongée.",
          "ergonomie": "La conception d'un produit pour qu'il soit confortable et efficace à utiliser.",
          "durabilité": "La capacité d'un produit à résister à l'usure ou à durer longtemps.",
          "esthétique": "L'apparence visuelle d'un produit et son attrait.",
          "compatibilité": "La capacité d'un produit ou d'un outil à fonctionner avec d'autres dispositifs ou logiciels.",
          "garantie": "Une promesse ou une politique d'une entreprise pour réparer ou remplacer un produit défectueux.",
          "options": "Des fonctionnalités ou caractéristiques supplémentaires qu'un produit peut offrir.",
          "design compact": "Un produit conçu pour être petit, léger et facilement transportable.",
          "packaging": "L'emballage ou la présentation extérieure d'un produit.",
          "efficacité": "La capacité d'un produit ou d'un outil à atteindre ses objectifs de manière optimale.",
          "praticité": "La facilité avec laquelle un produit peut être utilisé ou manipulé.",
          "durée de vie": "La période pendant laquelle un produit fonctionne correctement avant de devoir être remplacé.",
          "matériau": "La substance principale utilisée pour fabriquer un produit.",
          "écologique": "Un produit conçu pour minimiser son impact sur l'environnement.",

          # Technologies
          "logiciel": "Un programme informatique utilisé pour effectuer des tâches spécifiques.",
          "application": "Un programme conçu pour être utilisé sur un appareil comme un smartphone ou un ordinateur.",
          "mise à jour": "Une version améliorée d'un logiciel ou produit, souvent pour corriger des bugs ou ajouter des fonctionnalités.",
          "interface utilisateur": "La manière dont un utilisateur interagit avec un appareil ou un logiciel.",
          "intelligence artificielle": "La capacité des machines à simuler l'intelligence humaine pour accomplir des tâches.",
          "algorithme": "Une série d'instructions utilisées pour résoudre un problème ou effectuer un calcul.",
          "réalité augmentée": "Une technologie qui superpose des éléments virtuels sur le monde réel.",
          "réalité virtuelle": "Une expérience immersive créée dans un environnement numérique simulé.",
          "capteur": "Un composant utilisé pour détecter des changements physiques ou environnementaux.",
          "connectivité": "La capacité d'un appareil à se connecter à d'autres appareils ou réseaux.",
          "réseau": "Un système interconnecté permettant la communication entre appareils.",
          "cryptographie": "La science de la protection des informations en les rendant illisibles sans clé spécifique.",
          "big data": "Un ensemble massif de données qui nécessite des outils avancés pour être analysé.",
          "cloud computing": "L'utilisation de ressources informatiques via Internet au lieu de matériel local.",
          "open source": "Un logiciel dont le code source est librement accessible pour modification.",
          "protocole": "Un ensemble de règles définissant comment les appareils communiquent entre eux.",
          "bande passante": "La quantité maximale de données transférées sur un réseau en une seconde.",
          "robotique": "La conception et l'utilisation de robots pour accomplir des tâches automatisées.",
          "automatisation": "L'utilisation de systèmes pour effectuer des tâches sans intervention humaine directe.",

          # Services et expériences utilisateur
          "avis": random.choice(avis),
          "service client": "L'assistance fournie par une entreprise à ses clients pour résoudre des problèmes ou répondre à des questions.",
          "abonnement": "Un modèle de paiement récurrent pour accéder à un produit ou service.",
          "essai gratuit": "Une période pendant laquelle un produit ou service peut être testé sans coût.",
          "fidélité": "Le degré d'attachement d'un client à une marque ou entreprise.",
          "livraison": "Le processus d'envoi d'un produit au client.",
          "remboursement": "Le processus par lequel un client récupère de l'argent pour un produit ou service non satisfaisant.",
          "support technique": "L'assistance fournie pour résoudre les problèmes techniques liés à un produit.",
          "recommandation": "Un avis positif donné par un utilisateur ou un client à propos d'un produit ou d'un service.",
          "réputation": "L'image ou la perception générale d'une marque ou d'un produit auprès des consommateurs.",
          "communauté": "Un groupe d'utilisateurs partageant des idées, des solutions ou des expériences concernant un produit.",
          "fiabilité énergétique": "La capacité d'un appareil à consommer de l'énergie de manière efficace.",
          "expérience utilisateur": "La perception globale d'un utilisateur lorsqu'il interagit avec un produit ou service.",
          "installation": "Le processus de mise en place ou de configuration d'un produit pour son utilisation.",
          "satisfaction client": "Le niveau de contentement d'un client envers un produit ou service.",
          "politique de retour": "Les règles définies par une entreprise pour retourner ou échanger des produits.",

          # Expériences sociales et culturelles
          "confiance": "La croyance en la fiabilité ou l'honnêteté d'une personne, d'une marque ou d'un service.",
          "communication": "Le partage ou l'échange d'informations entre individus ou groupes.",
          "engagement": "La participation active ou l'implication d'un utilisateur ou client.",
          "collaboration": "Le travail en équipe pour produire un meilleur résultat.",
          "publicité": "La présentation d'un produit pour attirer des clients potentiels.",
          "marketing": "Les stratégies utilisées pour promouvoir un produit ou service.",
          "interaction": "L'échange entre un utilisateur et un produit, service ou communauté.",
          "tendance": "Une direction générale vers laquelle les préférences des consommateurs évoluent.",
          "partenariat": "Une collaboration entre deux entreprises pour atteindre un objectif commun.",
          "influenceur": "Une personne avec une large audience qui recommande des produits ou services.",
          "démographie": "Les caractéristiques d'un groupe cible, comme l'âge ou la localisation.",
          "personnalisation": "La capacité d'un produit ou service à être modifié selon les préférences d'un utilisateur.",
          "diversité culturelle": "La coexistence de différentes cultures et traditions au sein d'une société.",
          "réseaux sociaux": "Des plateformes en ligne permettant de partager du contenu et d'interagir avec d'autres utilisateurs.",
          "inclusion": "L'intégration et l'acceptation de tous, quelles que soient leurs différences.",
          "identité": "Le sentiment d'appartenance à un groupe culturel, social ou individuel.",
          "communauté": "Un groupe de personnes partageant des intérêts ou objectifs communs.",
          "engagement social": "L'implication active dans des initiatives ou causes sociales.",
          "accessibilité": "La facilité avec laquelle un produit, service ou lieu peut être utilisé par tous, y compris les personnes en situation de handicap.",
          "empathie": "La capacité de comprendre et de partager les sentiments d'autrui.",
          "influence culturelle": "L'impact des pratiques, croyances ou traditions d'une culture sur une autre.",

          # Produits supplémentaires
          "produit haut de gamme": "Un produit de qualité supérieure, souvent associé à une marque prestigieuse.",
          "produit recyclable": "Un produit fabriqué à partir de matériaux pouvant être réutilisés ou transformés après usage.",
          "produit en promotion": "Un produit vendu à un prix réduit pour une durée limitée.",
          "produit personnalisé": "Un produit adapté aux besoins ou préférences spécifiques d'un utilisateur.",
          "produit innovant": "Un produit introduisant une nouvelle fonctionnalité ou une amélioration significative.",
          "produit local": "Un produit fabriqué ou cultivé dans une région spécifique.",
          "produit d'entretien": "Un produit utilisé pour nettoyer ou maintenir un espace ou un objet.",
          "produit durable": "Un produit conçu pour durer longtemps tout en respectant l'environnement.",
          "produit importé": "Un produit fabriqué dans un autre pays et vendu localement.",

          # Argent et salaires
          "salaire brut": "La rémunération totale avant déductions comme les impôts et les cotisations sociales.",
          "salaire net": "La rémunération perçue par un employé après déduction des impôts et cotisations.",
          "revenu passif": "Un revenu généré sans intervention active continue, comme les investissements.",
          "pouvoir d'achat": "La capacité d'un individu ou d'un ménage à acheter des biens et services.",
          "budget": "Une estimation des revenus et des dépenses sur une période donnée.",
          "prime": "Une somme d'argent supplémentaire versée en récompense ou en complément du salaire.",
          "revenu mensuel": "La somme d'argent gagnée ou perçue sur une base mensuelle.",
          "investissement": "L'argent placé dans un projet ou un produit dans le but de générer un profit.",
          "intérêts": "Une somme d'argent payée pour l'emprunt de fonds ou gagnée sur un placement.",
          "épargne": "La partie non dépensée du revenu, souvent placée pour un usage futur.",

          # Jeux
          "jeu de société": "Un jeu nécessitant plusieurs participants, souvent autour d'un plateau ou de cartes.",
          "jeu en ligne": "Un jeu vidéo accessible via Internet, permettant souvent de jouer avec d'autres joueurs.",
          "jeu de stratégie": "Un jeu nécessitant de la planification et des décisions tactiques pour gagner.",
          "mode multijoueur": "Une fonctionnalité permettant à plusieurs joueurs de participer à un même jeu.",
          "jeu mobile": "Un jeu conçu pour être joué sur un smartphone ou une tablette.",
          "jeu gratuit (free-to-play)": "Un jeu accessible sans frais initiaux, souvent financé par des achats intégrés.",
          "niveaux": "Les étapes ou étapes progressives à franchir dans un jeu.",
          "DLC (contenu téléchargeable)": "Du contenu additionnel ajouté à un jeu existant moyennant un coût ou gratuit.",
          "skins": "Des personnalisations cosmiques pour un personnage ou un objet dans un jeu.",
          "e-sport": "Des compétitions organisées pour des jeux vidéo, souvent diffusées en ligne."
      }

    return definitions.get(idea.lower(), "Aucune définition disponible pour cette idée de produit.")

print(define_product_idea("produit facile à utiliser"))

Un produit où chaque fonctionnalité est présentée de manière logique et compréhensible.


initialize firebase

In [6]:
# Initialiser Firebase
def initialize_firebase(cert_path, db_url, app_name=None):
    if app_name and app_name in firebase_admin._apps:
        return firebase_admin.get_app(app_name)
    elif not app_name and firebase_admin._apps:
        return firebase_admin.get_app()
    cred = credentials.Certificate(cert_path)
    return firebase_admin.initialize_app(cred, {'databaseURL': db_url}, name=app_name)

generate product ideas

In [7]:
# Fonction pour générer des idées de produits à partir d'un commentaire
def generate_product_ideas(comment_text):
    product_ideas = []
    doc = nlp(comment_text)

    # Extraction des groupes nominaux
    for chunk in doc.noun_chunks:
        if chunk.root.pos_ in ("NOUN", "PROPN"):
            product_ideas.append(chunk.text)

    # Extraction des entités nommées
    for ent in doc.ents:
        if ent.label_ in ['PRODUCT', 'MISC', 'ORG']:
            product_ideas.append(ent.text)

    # Utilisation de BERT pour extraire des entités nommées
    inputs = tokenizer(comment_text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    logits = outputs.logits
    for idx, token in enumerate(inputs.input_ids[0]):
        if logits[0, idx, 1] > 0.5:
            product_ideas.append(tokenizer.decode(token, skip_special_tokens=True))

    # Ajout d'idées basées sur des expressions communes
    if "facile" in comment_text or "rapide" in comment_text or "confort" in comment_text:
        product_ideas.append("produit facile à utiliser")
    if "moins cher" in comment_text or "économique" in comment_text:
        product_ideas.append("produit économique")

    # Recherche de mots-clés spécifiques
    keywords = [
            'outil', 'appareil', 'technologie', 'produit', 'accessoire', 'marque',
            # Produits et objets
            'gadget', 'dispositif', 'matériel', 'équipement', 'instrument', 'module', 'composant', 'fourniture',
            'article', 'accessoire', 'consommable', 'outil numérique', 'interface', 'device', 'prototypage',

            # Marques et fabricants
            'constructeur', 'fournisseur', 'fabricant', 'distributeur', 'label', 'enseigne', 'modèle',
            'version', 'série', 'collection', 'édition', 'original', 'certification',

            # Utilisation et expérience
            'utilisateur', 'mode d’emploi', 'compatibilité', 'ergonomie', 'intuitif', 'personnalisation',
            'expérience', 'apprentissage', 'interface utilisateur', 'installation', 'adaptabilité', 'fiabilité',

            # Performances et caractéristiques
            'performance', 'qualité', 'durabilité', 'robustesse', 'résistance', 'polyvalence', 'innovation',
            'efficacité', 'productivité', 'fonctionnalité', 'vitesse', 'puissance', 'résolution', 'capacité',
            'autonomie', 'praticité', 'compact', 'léger', 'bruit', 'stabilité',

            # Aspects financiers
            'prix', 'valeur', 'remboursement', 'budget', 'abordable', 'coût', 'rentabilité', 'promotion',
            'soldes', 'rabais', 'investissement', 'économique', 'offre', 'marché',

            # Technologie et fonctionnalités
            'logiciel', 'application', 'intelligence artificielle', 'algorithme', 'machine learning',
            'réseau', 'cloud', 'big data', 'protocole', 'capteur', 'connectivité', 'bluetooth',
            'wifi', '5G', 'IOT', 'sécurité', 'encryption', 'crypto', 'API', 'driver', 'firmware',
            'bande passante', 'robotique', 'automatisation',

            # Esthétique et style
            'design', 'couleur', 'forme', 'texture', 'moderne', 'minimaliste', 'attrayant',
            'premium', 'classique', 'élégant', 'tendance', 'fashion', 'décoration',

            # Services et relations clients
            'service client', 'support technique', 'assistance', 'livraison', 'garantie', 'essai gratuit',
            'politique de retour', 'satisfaction', 'avis', 'recommandation', 'réputation', 'fidélité',
            'abonnement', 'contrat',

            # Jeux et divertissement
            'jeu', 'gaming', 'console', 'manette', 'casque', 'PC gamer', 'graphismes', 'FPS', 'RPG',
            'multijoueur', 'solo', 'réalisme', 'config', 'accessoires gaming', 'VR', 'streaming', 'téléchargement',

            # Écologie et durabilité
            'écologique', 'durable', 'recyclable', 'empreinte carbone', 'énergie renouvelable',
            'responsable', 'impact environnemental',

            # Problèmes et critiques
            'bug', 'défaut', 'problème', 'retard', 'dommage', 'erreur', 'dysfonctionnement', 'réparation',
            'incompatibilité', 'critique', 'faible', 'déception', 'contre-indication',

            # Communauté et tendances
            'discussion', 'suggestion', 'avis', 'feedback', 'note', 'astuce', 'amélioration', 'conseil',
            'partage', 'tutoriel', 'expérience utilisateur', 'top', 'tendance', 'mode', 'découverte',
            'innovation'
        ]

    for word in comment_text.split():
        if word.lower() in keywords:
            product_ideas.append(f"produit de type {word}")

    return product_ideas


In [8]:
# REDDIT FR
# certificate_path_source = '/content/redditproblemdatascrapefr-firebase-adminsdk-z3iex-8c9ddd6f6d.json'
# database_url_source = userdata.get('database_url')

# source_app = initialize_firebase(
#     certificate_path_source,
#     database_url_source,
#     'source_app'
# )

# certificate_path_target = '/content/redditdatascrapekeywords-firebase-adminsdk-wfyx4-ee94cdfc6b.json'
# database_url_target = userdata.get('database_url_target')

# target_app = initialize_firebase(
#     certificate_path_target,
#     database_url_target,
#     'target_app'
# )

# TWITTER
certificate_path_source = '/content/twitterdatascrape-firebase-adminsdk-jrsqj-4680934511.json'
database_url_source = userdata.get('db_url_twitter_src')

source_app = initialize_firebase(
    certificate_path_source,
    database_url_source,
    'source_app'
)

certificate_path_target = '/content/twitterdatascrapekeywords-firebase-adminsdk-gitdj-3bc4f2b4a1.json'
database_url_target = userdata.get('db_url_twitter_key')

target_app = initialize_firebase(
    certificate_path_target,
    database_url_target,
    'target_app'
)



In [9]:
source_database_reference = db.reference(app=source_app)
target_database_reference = db.reference(app=target_app)

In [18]:
# Charger le modèle SpaCy pour l'analyse linguistique
# nlp = spacy.load("fr_core_news_sm")
nlp = spacy.load("en_core_web_sm")

# Charger le modèle BERT pré-entraîné pour la classification des tokens
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForTokenClassification.from_pretrained('bert-base-uncased')

# Fonction pour nettoyer les clés Firebase
def sanitize_firebase_key(key):
    return re.sub(r'[./\[\]#$\u0000-\u001F\u007F]', '_', key).strip()

# !!! those three lines of code are very important for the recuperation post id !!!
# posts = db.reference('2025-01-23/', app=source_app).get()
posts = db.reference('tweets_try3 - 2024-09-17/', app=source_app).get()
all_posts = []
for post in posts:
  # print("==========================> post id: ",post)
  all_posts.append(post["full_text"])

# post = []
# for p in all_posts:
#     # Convert dict_keys to a list before accessing elements by index
#     keys_list = list(p.keys())
#     if len(keys_list) > 2: # Check if there are at least 3 keys
#       print("---------> ", keys_list[2])
#       post.append(keys_list[2])
#     else:
#       print("---------> This dictionary has fewer than 3 keys.")

# Charger les commentaires depuis Firebase
# comments = db.reference(f'2025-01-23/{all_posts[0]}/comments', app=source_app).get()
comments = all_posts


# print("all comments that i have ----------------> ", comments)

# if not comments:
#     print("Aucun commentaire trouvé.")
#     exit()

all_texts = comments

# Clustering des commentaires
comment_vectors = np.array([nlp(comment).vector for comment in all_texts])
clusters = defaultdict(list)
if comment_vectors.size > 0:
    num_clusters = 5
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(comment_vectors)
    for idx, label in enumerate(kmeans.labels_):
        clusters[label].append(all_texts[idx])
else:
    print("Impossible de réaliser le clustering, aucun vecteur de commentaire trouvé.")

# Regrouper et sauvegarder les idées dans Firebase
clustered_product_ideas = defaultdict(list)
for cluster_id, cluster_comments in clusters.items():
    for comment in cluster_comments:
        ideas = generate_product_ideas(comment)
        clustered_product_ideas[f"Cluster {cluster_id + 1}"].extend(ideas)

        # Change to: Assuming 'id' is a key in your post data
        # clustered_product_ideas[f"Cluster {cluster_id + 1} {all_posts[0].get('id', 'default_id')}"].extend(ideas)



current_date_str = f"x-fitness-{datetime.date.today()}"
for cluster, ideas in clustered_product_ideas.items():
    sanitized_cluster = sanitize_firebase_key(cluster)
    unique_ideas = list(set(ideas))
    sanitized_ideas = [
        {
            "idée": idea,
            "définition": define_product_idea(idea),
            "clé_nettoyée": sanitize_firebase_key(idea)
        }
        for idea in unique_ideas
    ]

    target_database_reference.child(f"{current_date_str}").set({"date": f"{current_date_str}", "frequency_table": sanitized_ideas})

# Afficher les idées de produits regroupées
for cluster, ideas in clustered_product_ideas.items():
    print(f"{cluster}:")
    for idea in set(ideas):
        print(f"  Produit suggéré: {idea} - {define_product_idea(idea)}")


/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Cluster 4:
  Produit suggéré: ContentCreator - Aucune définition disponible pour cette idée de produit.
  Produit suggéré: i t - Aucune définition disponible pour cette idée de produit.
  Produit suggéré: 👏
شروط المسابقة - Aucune définition disponible pour cette idée de produit.
  Produit suggéré: ボディビルダーのイリア「ゴーレム」イェフィムチクが死去 - Aucune définition disponible pour cette idée de produit.
  Produit suggéré: s t a r t - Aucune définition disponible pour cette idée de produit.
  Produit suggéré: #SaturdayThought - Aucune définition disponible pour cette idée de produit.
  Produit suggéré: 2- منشن ل3 أشخاص - Aucune définition disponible pour cette idée de produit.
  Produit suggéré: MondayVibes - Aucune définition disponible pour cette idée de produit.
  Produit suggéré: MondayMotivaton - Aucune définition disponible pour cette idée de produit.
  Produit suggéré: fitness https://t.co/0DchIKIVaZ - Aucune définition disponible pour cette idée de produit.
  Produit suggéré: اليوم - Aucune définiti